Task 2.2: Getting Started with tweepy 

In [1]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'apVFqg1nQzntPQRtfnCcnKazl'
consumer_secret = ''
access_token = '831480969823715328-mdG8bdkqmfjGEtXSbER5FMAeHEOQ6eg'
access_secret = 'x4PeYShmCWcRL9xMbpx5BEjv8k5BovvJZx67HjGYUrBhp'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

user = api.me()
 
print('Name: ' + user.name)
print('Location: ' + user.location)
print('Friends: ' + str(user.followers_count))
print('Created: ' + str(user.created_at))
print('Description: ' + str(user.description))

Name: Hao Wu
Location: 
Friends: 0
Created: 2017-02-14 12:31:26
Description: 


Task 2.2.3: Accessing Tweets  

In [2]:
import json

for status in tweepy.Cursor(api.home_timeline).items(1):
    print(json.dumps(status._json, indent=2))


{
  "contributors": null, 
  "truncated": false, 
  "text": "RT @TheVertical: With a core led by Gordon Hayward and Rudy Gobert, the Jazz are back. @WojVerticalNBA column on @TheVertical. https://t.co\u2026", 
  "is_quote_status": false, 
  "in_reply_to_status_id": null, 
  "id": 859078245169459204, 
  "favorite_count": 0, 
  "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
  "retweeted": false, 
  "coordinates": null, 
  "entities": {
    "symbols": [], 
    "user_mentions": [
      {
        "id": 432570714, 
        "indices": [
          3, 
          15
        ], 
        "id_str": "432570714", 
        "screen_name": "TheVertical", 
        "name": "The Vertical"
      }, 
      {
        "id": 50323173, 
        "indices": [
          87, 
          102
        ], 
        "id_str": "50323173", 
        "screen_name": "WojVerticalNBA", 
        "name": "Adrian Wojnarowski"
      }, 
      {
        "id": 432570714, 
        "i

In [2]:
for friend in tweepy.Cursor(api.friends).items(1):
    print(json.dumps(friend._json, indent=2))


NameError: name 'tweepy' is not defined

In [8]:
for tweet in tweepy.Cursor(api.user_timeline).items(1):
    print(json.dumps(tweet._json, indent=2))

{
  "contributors": null, 
  "truncated": false, 
  "text": "\u00a1Hola, Twitter! #miprimerTweet", 
  "is_quote_status": false, 
  "in_reply_to_status_id": null, 
  "id": 859086168260050945, 
  "favorite_count": 0, 
  "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", 
  "retweeted": false, 
  "coordinates": null, 
  "entities": {
    "symbols": [], 
    "user_mentions": [], 
    "hashtags": [
      {
        "indices": [
          16, 
          30
        ], 
        "text": "miprimerTweet"
      }
    ], 
    "urls": []
  }, 
  "in_reply_to_screen_name": null, 
  "in_reply_to_user_id": null, 
  "retweet_count": 0, 
  "id_str": "859086168260050945", 
  "favorited": false, 
  "user": {
    "follow_request_sent": false, 
    "has_extended_profile": false, 
    "profile_use_background_image": true, 
    "default_profile_image": true, 
    "id": 831480969823715328, 
    "profile_background_image_url_https": null, 
    "verified": false, 
    "translator_t

In [9]:
from nltk.tokenize import word_tokenize

tweet = 'RT @JordiTorresBCN: just an example! :D http://JordiTorres.Barcelona #masterMEI'

print(word_tokenize(tweet))

['RT', '@', 'JordiTorresBCN', ':', 'just', 'an', 'example', '!', ':', 'D', 'http', ':', '//JordiTorres.Barcelona', '#', 'masterMEI']


In [10]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @JordiTorresBCN: just an example! :D http://JordiTorres.Barcelona #masterMEI'
print(preprocess(tweet))

['RT', '@JordiTorresBCN', ':', 'just', 'an', 'example', '!', ':D', 'http://JordiTorres.Barcelona', '#masterMEI']
